In [0]:
%%capture
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl 
!pip3 install torchvision

from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch
!pip install torchtext
!pip install spacy && python -m spacy download en

In [0]:
import pandas as pd
labeled_train = "https://raw.githubusercontent.com/kj2013/claff-happydb/master/data/TRAIN/labeled_10k.csv"
unlabeled_train = "https://raw.githubusercontent.com/kj2013/claff-happydb/master/data/TRAIN/unlabeled_70k.csv"
unlabeled_test = "https://raw.githubusercontent.com/kj2013/claff-happydb/master/data/TEST/unlabeled_17k.csv"
labeled_data = pd.read_table(labeled_train,sep=',',index_col=False, error_bad_lines=False) 

In [3]:
labeled_data.head(10)

,hmid,moment,concepts,agency,social,age,country,gender,married,parenthood,reflection,duration
0,27674,I was happy when my son got 90% marks in his e...,education|family,no,yes,29.0,IND,m,married,y,24h,half_a_day
1,27685,went to movies with my friends it was fun,entertainment,yes,yes,29.0,IND,m,single,y,24h,half_a_day
2,27691,A hot kiss with my girl friend last night made...,romance,yes,yes,25.0,IND,m,married,y,24h,at_least_one_hour
3,27701,My son woke me up to a fantastic breakfast of ...,family|food,no,yes,79,USA,f,widowed,y,24h,all_day_im_still_feeling_it
4,27712,My older daughter keeps patting my younger dau...,family,no,yes,30,USA,f,married,y,24h,a_few_moment
5,27737,I cooked my girlfriend a wonderful breakfast.,food|romance,yes,yes,34,USA,m,single,n,24h,half_a_day
6,27754,My Mother gave me a surprise visit at my home.,family,no,yes,26,IND,m,married,n,24h,half_a_day
7,27772,There was hardly any traffic on my way to work...,career,yes,no,25,USA,m,single,n,24h,at_least_one_hour
8,27775,I came to my office at right time.,career,yes,no,22,USA,m,single,n,24h,a_few_moment
9,27777,The day I got my degree in industrial engineering,education,yes,no,22,VEN,f,single,n,24h,all_day_im_still_feeling_it


In [4]:
len(labeled_data.country.unique())

70

In [5]:
len(labeled_data.concepts.unique())

294

In [0]:
import numpy as np
from sklearn.model_selection import train_test_split
train, valid = train_test_split(labeled_data, test_size=0.2, random_state=0, stratify=labeled_data['social'])

In [7]:
train.social.value_counts()

yes    4500
no     3948
Name: social, dtype: int64

In [8]:
valid.social.value_counts()

yes    1125
no      987
Name: social, dtype: int64

In [0]:
train = train[['hmid','moment','agency']]
valid = valid[['hmid','moment','agency']]

In [10]:
train.head(10)

,hmid,moment,agency
484,32335,"I went to an art gallery, which I've never don...",yes
217,29833,I learned how to cook a new cake recipe.,yes
3608,62136,All my friends give me surprise party on my bi...,no
7568,100495,I ordered three pairs of pants online and when...,yes
1805,44630,I was able to successfully clean a student's c...,yes
2218,48616,I watched one of my favorite shows after a hia...,yes
1318,40100,Gonzaga won its Final Four basketball game.,no
2820,54298,I talked on the phone with a good friend of mi...,yes
4891,74912,Going to get a coffee at the bookstore and bro...,yes
4627,72216,Watched a friend's daughter play soccer,yes


In [0]:
train.to_csv('train_data.csv',index=False)
valid.to_csv('valid_data.csv',index=False)

In [12]:
import torch 
import torch.nn as nn
import torchvision.datasets as datasets
from skimage import transform
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.optim as optim
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from collections import Counter,OrderedDict
from keras.utils.vis_utils import *
import random
import math

Using TensorFlow backend.


In [0]:
%%capture
from torchtext import vocab
# specify the path to the localy saved vectors
vec = vocab.GloVe(name='840B', dim=300)
# vec = vocab.GloVe(name='6B', dim=200)

In [0]:
import torchtext
import spacy
import re
from torchtext import data

# tokenizer function using spacy
nlp = spacy.load('en',disable=['parser', 'tagger', 'ner'])
def tokenizer(s): 
    return [w.text.lower() for w in nlp(tweet_clean(s))]

def tweet_clean(text):
    text = re.sub(r'[^A-Za-z0-9]+', ' ', text) # remove non alphanumeric character
    text = re.sub(r'https?:/\/\S+', ' ', text) # remove links
    return text.strip()

def tokenizer(s): 
    return [w.text.lower() for w in nlp(tweet_clean(s))]

# define the columns that we want to process and how to process
TEXT = data.Field(sequential=True, 
                 tokenize=tokenizer, 
                 include_lengths=True, 
                 use_vocab=True)
LABEL = data.Field(sequential=False, 
                   use_vocab=True,
                   pad_token=None, 
                   unk_token=None)

train_val_fields = [
    ("hmid", None),
    ('moment', TEXT), # process it as text
    ('agency', LABEL) # process it as label
]

trainds, valds = data.TabularDataset.splits(path='', 
                                            format='csv', 
                                            train='train_data.csv', 
                                            validation='valid_data.csv', 
                                            fields=train_val_fields, 
                                            skip_header=True)

In [0]:
# build the vocabulary using train and validation dataset and assign the vectors
TEXT.build_vocab(trainds,valds, max_size=100000, vectors=vec)
# build vocab for labels
LABEL.build_vocab(trainds)

In [16]:
print(LABEL.vocab.vectors)

None


In [0]:
traindl, valdl = data.BucketIterator.splits(datasets=(trainds, valds), # specify train and validation Tabulardataset
                                            batch_sizes=(64,len(valid)),  # batch size of train and validation
                                            sort_key=lambda x: len(x.moment), # on what attribute the text should be sorted
                                            device=None, # -1 mean cpu and 0 or None mean gpu
                                            sort_within_batch=True, 
                                            repeat=False)

In [18]:
print(len(traindl), len(valdl))

132 1


In [19]:
traindl

In [0]:
batch = next(iter(traindl)) # BucketIterator return a batch object

In [22]:
print(batch.agency) # labels of the batch

tensor([0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1,
        1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0])


In [23]:
print(batch.moment) 

(tensor([[   2,    2, 6417,  ...,    2,    2,  516],
        [ 433,   83,   15,  ...,   85,    8,    3],
        [   9,    4,   62,  ...,    4,  795,   40],
        ...,
        [   2,  590, 2492,  ...,    8,    3,   22],
        [  17,   36,   61,  ...,   49, 1082,   10],
        [  15,   26,  169,  ...,  247,   59,   16]]), tensor([17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
        17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
        17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
        17, 17, 17, 17, 17, 17, 17, 17, 17, 17]))


In [24]:
print(batch.dataset.fields)

{'hmid': None, 'moment': <torchtext.data.field.Field object at 0x7f51ddf9c6d8>, 'agency': <torchtext.data.field.Field object at 0x7f5271bb9b70>}


In [25]:
(x,lengths) = batch.moment
x.shape

torch.Size([17, 64])

In [26]:
batch.agency.shape

torch.Size([64])

In [0]:
class BatchGenerator:
    def __init__(self, dl, x_field, y_field):
        self.dl, self.x_field, self.y_field = dl, x_field, y_field
        
    def __iter__(self):
        for batch in self.dl:
            X = getattr(batch, self.x_field)
            y = getattr(batch, self.y_field)
            yield (X,y)
 


In [0]:
train_batch_it = BatchGenerator(traindl, 'moment', 'agency')
valid_batch_it = BatchGenerator(valdl, 'moment', 'agency')

In [0]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
 
class LSTM(nn.Module):
    def __init__(self, hidden_dim, n_lstm_layers,emb_dim, dropout_p):
        super(LSTM,self).__init__() # don't forget to call this!
        self.embedding = nn.Embedding(len(TEXT.vocab), emb_dim)
        self.embedding.weight = nn.Parameter(TEXT.vocab.vectors, requires_grad=False)
        self.encoder = nn.LSTM(emb_dim, hidden_dim, num_layers=n_lstm_layers,dropout=dropout_p)
        self.dropout = nn.Dropout(dropout_p)
        self.fc1=nn.Linear(hidden_dim,64)
        self.predictor=nn.Linear(64,1)
 
    def forward(self, seq):
        hdn, _ = self.encoder(self.embedding(seq))
        output = hdn[-1, :, :]
        fc_output= self.fc1(output)
        fc_output= self.dropout(fc_output)
        preds=self.predictor(fc_output)
        return torch.sigmoid(preds)

In [0]:
embed_size = 300
nh = 128
drop=0.75
n_lstm_layers=2
model = LSTM(nh,n_lstm_layers,embed_size,drop)

In [31]:
from tqdm import tqdm
from sklearn.metrics import accuracy_score
from torch.optim.lr_scheduler import StepLR
opt = optim.Adam(model.parameters(), lr=1e-3)
loss_func = nn.BCELoss()
epochs = 20
batch_size = 64 
scheduler = StepLR(opt, step_size=5, gamma=0.1)

for epoch in range(1, epochs + 1):
    running_loss = 0.0
    running_corrects = 0
    iterations=0
    scheduler.step()
    model.train() # turn on training mode
    for x,y in tqdm(train_batch_it,disable=True): # thanks to our wrapper, we can intuitively iterate over our data!
        iterations=iterations+1
        opt.zero_grad()
        (x,lengths)=x
        predicted = model(x)
        predicted=predicted.view(-1)
        loss = loss_func(predicted,y.float())
        loss.backward()
        opt.step()
        running_loss += loss.item() * x.size(0)
    epoch_loss = running_loss / len(trainds)
 
    # calculate the validation loss for this epoch
    val_loss = 0.0
    list_scores=[]
    model.eval() # turn on evaluation mode
    for x,y in tqdm(valid_batch_it,disable=True):
        (x,lengths)=x
        preds = model(x)
        loss = loss_func(preds,y.float())
        val_loss += loss.item() * x.size(0)
        preds = [int(x>0.5) for x in preds]
        temp=accuracy_score(preds,y)
    
    val_loss /= len(valds)
    state = {
    'epoch': epoch,
    'state_dict': model.state_dict(),
    'optimizer': opt.state_dict(),
    }
    torch.save(state, 'mytraining1'+str(epoch)+'.pt')
    print('Epoch: {}, Training Loss: {:.4f}, Validation Loss: {:.4f}, Validation Accuracy: {:.4f}'.format(epoch, epoch_loss, val_loss,temp))


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1594: UserWarning: Using a target size (torch.Size([2112])) that is different to the input size (torch.Size([2112, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


Epoch: 1, Training Loss: 0.1204, Validation Loss: 0.0163, Validation Accuracy: 0.7382
Epoch: 2, Training Loss: 0.0959, Validation Loss: 0.0202, Validation Accuracy: 0.8333
Epoch: 3, Training Loss: 0.0987, Validation Loss: 0.0147, Validation Accuracy: 0.8423
Epoch: 4, Training Loss: 0.0843, Validation Loss: 0.0154, Validation Accuracy: 0.8281
Epoch: 5, Training Loss: 0.0826, Validation Loss: 0.0196, Validation Accuracy: 0.7760
Epoch: 6, Training Loss: 0.0740, Validation Loss: 0.0247, Validation Accuracy: 0.4252
Epoch: 7, Training Loss: 0.0734, Validation Loss: 0.0239, Validation Accuracy: 0.4740
Epoch: 8, Training Loss: 0.0728, Validation Loss: 0.0292, Validation Accuracy: 0.3438
Epoch: 9, Training Loss: 0.0713, Validation Loss: 0.0213, Validation Accuracy: 0.6089
Epoch: 10, Training Loss: 0.0691, Validation Loss: 0.0211, Validation Accuracy: 0.6373
Epoch: 11, Training Loss: 0.0699, Validation Loss: 0.0219, Validation Accuracy: 0.6075
Epoch: 12, Training Loss: 0.0693, Validation Loss: 0

In [0]:
index = 3
from google.colab import files
files.download('mytraining'+str(3)+'.pt')